# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 05**: Data pipeline with Neo4j

**Date**: October 2nd 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

# Dataset description

# Data ingestion

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages",
"org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.neo4j#neo4j-connector-apache-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d946f15e-a09b-48fa-b14a-20a35f1eed5d;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.13;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.13_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in central
	found org.neo4j#caniuse-api;1.3.0 in central
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in central
	found org.jetbrains#annotations;13.0 in central
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in central
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found io.netty#netty-handler;4.1.

In [2]:
from regalado_floriano.spark_utils import  SparkUtils
#https://www.kaggle.com/datasets/anshulmehtakaggl/github-organizations-social-network-analysis
path = "/opt/spark/work-dir/data/github_organizations/"
org_schema = SparkUtils.generate_schema([("Organization", "string"), ("member", "string")])

df_orgs = spark.read.option("header", "true")\
                .schema(org_schema)\
                .csv(path + "organization.csv")

In [3]:
df_orgs.show()

[Stage 0:>                                                          (0 + 1) / 1]

+------------+--------------------+
|Organization|              member|
+------------+--------------------+
|    ethereum|       Oleh Aldekein|
|    ethereum|                None|
|    ethereum|  Frank Szendzielarz|
|    ethereum|   Kolby Moroz Liebl|
|    ethereum|Marius van der Wi...|
|    ethereum|        Mihai Alisie|
|    ethereum|                None|
|    ethereum|    Anton Nashatyrev|
|    ethereum|       Justin Martin|
|    ethereum|         RJ Catalano|
|    ethereum| Yoshitomo Nakanishi|
|    ethereum|      Alexander Arlt|
|    ethereum|   Alessandro Coglio|
|    ethereum|                acud|
|    ethereum|  Patricio Palladino|
|    ethereum|           Angela Lu|
|    ethereum|    Alex Beregszaszi|
|    ethereum|                None|
|    ethereum|               becca|
|    ethereum|        Kamil Śliwak|
+------------+--------------------+
only showing top 20 rows


# Transformations

In [13]:
# Add the code for your transformations to create nodes and edges DataFrames HERE
#there really isn't a lot to do. it is already ready to be imported
from pyspark.sql.functions import col,monotonically_increasing_id

df_orgs  = df_orgs.select(col("Organization").alias("Organization"),col("member").alias("Member"))
organizations = df_orgs.select(col("Organization").alias("name")).distinct().withColumn("id",monotonically_increasing_id())
users = df_orgs.select(col("member").alias("name")).distinct().withColumn("id",monotonically_increasing_id()) 

# Writing Data in Neo4j

In [17]:
# Add the code to write a graph from PySpark's DataFrames to Neo4j
neo4j_url = "bolt://neo4j-iteso:7687"
neo4j_user = "neo4j"
neo4j_passwd = "neo4j@1234"
#I will include this in spark utils, with the class instanstiationg the user and url and passwd
def loadNodes(df, name, key):
    df.write \
      .format("org.neo4j.spark.DataSource") \
      .mode("Overwrite") \
      .option("url", neo4j_url) \
      .option("authentication.basic.username", neo4j_user) \
      .option("authentication.basic.password", neo4j_passwd) \
      .option("labels", name) \
      .option("node.keys", key) \
      .save()

def loadBRel(column, rel_name, src,dst):
    column.write \
      .format("org.neo4j.spark.DataSource") \
      .mode("Overwrite") \
      .option("url", neo4j_url) \
      .option("authentication.basic.username", neo4j_user) \
      .option("authentication.basic.password", neo4j_passwd) \
      .option("relationship", rel_name) \
      .option("relationship.save.strategy", "keys") \
      .option("relationship.source.labels", ":" + src) \
      .option("relationship.source.save.mode", "match") \
      .option("relationship.source.node.keys", f"{src}:name") \
      .option("relationship.target.labels", ":" + dst ) \
      .option("relationship.target.save.mode", "match") \
      .option("relationship.target.node.keys", f"{dst}:name") \
      .save()

    
def readCypher(query):
    return spark.read \
    .format("org.neo4j.spark.DataSource") \
    .option("url", neo4j_url) \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_passwd) \
    .option("query",query)\
    .load()
    

In [8]:

loadNodes(organizations, ":Organization", "id")
loadNodes(users, ":Member", "id")

In [18]:
loadBRel(df_orgs, "BELONGS_TO", "Member", "Organization")

# Read and Query Graphs with PySpark

In [ ]:
# Add the code to read a data frame from Neo4J and run a simple query to verify 

In [16]:
readCypher("Match (m)-[B:BELONGS_TO]->(o) return m.name, o.name").show(10)

+------+------+
|m.name|o.name|
+------+------+
+------+------+



In [ ]:
sc.stop()